<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/SN_web_lightmode.png" width="300">
</center>


<h1>Analysis of Global COVID-19 Pandemic Data</h1>

Estimated time needed: **90** minutes



## Overview:

There are 10 tasks in this final project. All tasks will be graded by your peers who are also completing this assignment within the same session.

You need to submit the following the screenshot for the code and output for each task for review.

If you need to refresh your memories about specific coding details, you may refer to previous hands-on labs for code examples.


In [ ]:
# If you are working on your local Jupyter notebook, please uncomment the below code and install the packages

#install.packages("httr")
#install.packages("rvest")

In [1]:
#require("httr")
#require("rvest")

library(httr)
library(rvest)

Loading required package: xml2


Note: if you can import above libraries, please use install.packages() to install them first.


## TASK 1: Get a `COVID-19 pandemic` Wiki page using HTTP request


First, let's write a function to use HTTP request to get a public COVID-19 Wiki page.

Before you write the function, you can open this public page from this 

URL https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country using a web browser.

The goal of task 1 is to get the html page using HTTP request (`httr` library)


In [2]:

get_wiki_covid19_page <- function() {
    
  # Our target COVID-19 wiki page URL is: https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country  
  # Which has two parts: 
    # 1) base URL `https://en.wikipedia.org/w/index.php  
    # 2) URL parameter: `title=Template:COVID-19_testing_by_country`, seperated by question mark ?
    
  # Wiki page base
  wiki_base_url <- "https://en.wikipedia.org/w/index.php"
  # You will need to create a List which has an element called `title` to specify which page you want to get from Wiki
  # in our case, it will be `Template:COVID-19_testing_by_country`
    Template_par <- list(title = 'Template:COVID-19_testing_by_country')
 
  # - Use the `GET` function in httr library with a `url` argument and a `query` arugment to get a HTTP response
    wiki_res <- GET(wiki_base_url, query = Template_par)
    
  # Use the `return` function to return the response
    return(wiki_res)

}


Call the `get_wiki_covid19_page` function to get a http response with the target html page


In [3]:
# Call the get_wiki_covid19_page function and print the response
get_wiki_covid19_page_res <- get_wiki_covid19_page()
print(get_wiki_covid19_page_res)

Response [https://en.wikipedia.org/w/index.php?title=Template%3ACOVID-19_testing_by_country]
  Date: 2023-02-16 09:01
  Status: 200
  Content-Type: text/html; charset=UTF-8
  Size: 425 kB
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-fea...
<head>
<meta charset="UTF-8"/>
<title>Template:COVID-19 testing by country - Wikipedia</title>
<script>document.documentElement.className="client-js vector-feature-language...
"wgCSPNonce":false,"wgCanonicalNamespace":"Template","wgCanonicalSpecialPageN...
"CS1 Khmer-language sources (km)","CS1 uses Chinese-language script (zh)","CS...
"CS1 foreign language sources (ISO 639-2)","CS1 Burmese-language sources (my)...
"pageLanguageCode":"en","pageLanguageDir":"ltr","pageVariantFallbacks":"en"},...
...


## TASK 2: Extract COVID-19 testing data table from the wiki HTML page


On the COVID-19 testing wiki page, you should see a data table `<table>` node contains COVID-19 testing data by country on the page:

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0101ENCoursera889-2022-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/covid-19-by-country.png" width="400" align="center">
</a>

Note the numbers you actually see on your page may be different from above because it is still an on-going pandemic when creating this notebook.

The goal of task 2 is to extract above data table and convert it into a data frame


Now use the `read_html` function in rvest library to get the root html node from response


In [4]:
# Get the root html node from the http response in task 1 
covid19_root_node <- read_html(get_wiki_covid19_page_res)

Get the tables in the HTML root node using `html_nodes` function.


In [5]:
# Get the table node from the root html node
covid19_table_node <- html_node(covid19_root_node, "table")

Read the specific table from the multiple tables in the table node using the `html_table` function and convert it into dataframe using `as.data.frame`


In [6]:
# Read the table node and convert it into a data frame, and print the data frame for review
covid_data_frame <- html_table(covid19_table_node)
head(covid_data_frame)

,X1,X2
,<lgl>,<chr>
1,NA,This template needs to be updated. Please help update this template to reflect recent events or newly available information. Relevant discussion may be found on the talk page.


## TASK 3: Pre-process and export the extracted data frame

The goal of task 3 is to pre-process the extracted data frame from the previous step, and export it as a csv file


Let's get a summary of the data frame


In [44]:
# Print the summary of the data frame
summary(covid_data_frame)

    X1               X2           
 Mode:logical   Length:1          
 NA's:1         Class :character  
                Mode  :character  

As you can see from the summary, the columns names are little bit different to understand and some column data types are not correct. For example, the `Tested` column shows as `character`. 

As such, the data frame read from HTML table will need some pre-processing such as removing irrelvant columns, renaming columns, and convert columns into proper data types.


We have prepared a pre-processing function for you to conver the data frame but you can also try to write one by yourself


In [17]:
preprocess_covid_data_frame <- function(data_frame) {
    
    shape <- dim(data_frame)

    # Remove the World row
    data_frame<-data_frame[!(data_frame$`Country or region`=="World"),]
    # Remove the last row
    data_frame <- data_frame[1:172, ]
    
    # We dont need the Units and Ref columns, so can be removed
    data_frame["Ref."] <- NULL
    data_frame["Units[b]"] <- NULL
    
    # Renaming the columns
    names(data_frame) <- c("country", "date", "tested", "confirmed", "confirmed.tested.ratio", "tested.population.ratio", "confirmed.population.ratio")
    
    # Convert column data types
    data_frame$country <- as.factor(data_frame$country)
    data_frame$date <- as.factor(data_frame$date)
    data_frame$tested <- as.numeric(gsub(",","",data_frame$tested))
    data_frame$confirmed <- as.numeric(gsub(",","",data_frame$confirmed))
    data_frame$'confirmed.tested.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.tested.ratio`))
    data_frame$'tested.population.ratio' <- as.numeric(gsub(",","",data_frame$`tested.population.ratio`))
    data_frame$'confirmed.population.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.population.ratio`))
    
    return(data_frame)
}



Call the `preprocess_covid_data_frame` function


In [18]:
# call `preprocess_covid_data_frame` function and assign it to a new data frame
new_covid_data_frame <- preprocess_covid_data_frame(covid_data_frame)
head(new_covid_data_frame)

ERROR: Error in names(data_frame) <- c("country", "date", "tested", "confirmed", : 'names' attribute [7] must be the same length as the vector [2]


Get the summary of the processed data frame again


In [45]:
# Print the summary of the processed data frame again
summary(new_covid_data_frame)

                country             date         tested         
 Afghanistan        :  1   2 Feb 2023 :  7   Min.   :     3880  
 Albania            :  1   1 Feb 2023 :  4   1st Qu.:   512037  
 Algeria            :  1   31 Jan 2023:  4   Median :  3029859  
 Andorra            :  1   1 Mar 2021 :  3   Mean   : 31348974  
 Angola             :  1   23 Jul 2021:  3   3rd Qu.: 12386725  
 Antigua and Barbuda:  1   29 Jan 2023:  3   Max.   :929349291  
 (Other)            :166   (Other)    :148                      
   confirmed        confirmed.tested.ratio tested.population.ratio
 Min.   :       0   Min.   : 0.00          Min.   :   0.006       
 1st Qu.:   37839   1st Qu.: 5.00          1st Qu.:   9.475       
 Median :  281196   Median :10.05          Median :  46.950       
 Mean   : 2506926   Mean   :11.25          Mean   : 175.471       
 3rd Qu.: 1278105   3rd Qu.:15.25          3rd Qu.: 156.500       
 Max.   :90749469   Max.   :46.80          Max.   :3223.000       
           

After pre-processing, you can see the columns and columns names are simplified, and columns types are converted into correct types.


The data frame has following columns:

- **country** - The name of the country
- **date** - Reported date
- **tested** - Total tested cases by the reported date
- **confirmed** - Total confirmed cases by the reported date
- **confirmed.tested.ratio** - The ratio of confirmed cases to the tested cases
- **tested.population.ratio** - The ratio of tested cases to the population of the country
- **confirmed.population.ratio** - The ratio of confirmed cases to the population of the country


OK, we can call `write.csv()` function to save the csv file into a file. 


In [55]:
# Export the data frame to a csv file
write.csv(new_covid_data_frame, 'covid.csv')

Note for IBM Waston Studio, there is no traditional "hard disk" associated with a R workspace.

Even if you call `write.csv()` method to save the data frame as a csv file, it won't be shown in IBM Cloud Object Storage asset UI automatically.

However, you may still check if the `covid.csv` exists using following code snippet:


In [59]:
# Get working directory
wd <- getwd()
# Get exported 
file_path <- paste(wd, sep="", "/covid.csv")
# File path
print(file_path)
file.exists(file_path)

[1] "/resources/v2/M5_Final/covid.csv"


[1] TRUE

**Optional Step**: If you have difficulties finishing above webscraping tasks, you may still continue with next tasks by downloading a provided csv file from here:


In [ ]:
## Download a sample csv file
# covid_csv_file <- download.file("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/dataset/covid.csv", destfile="covid.csv")
# covid_data_frame_csv <- read.csv("covid.csv", header=TRUE, sep=",")

## TASK 4: Get a subset of the extracted data frame

The goal of task 4 is to get the 5th to 10th rows from the data frame with only `country` and `confirmed` columns selected


In [62]:
# Read covid_data_frame_csv from the csv file
covid_df_csv <- read.csv("covid.csv", header =TRUE, sep=",")
# Get the 5th to 10th rows, with two "country" "confirmed" columns
covid_df_row <- covid_df_csv[c(5:10),c("country","confirmed")]
covid_df_row

,country,confirmed
,<fct>,<int>
5,Angola,20981
6,Antigua and Barbuda,832
7,Argentina,9060495
8,Armenia,422963
9,Australia,10112229
10,Austria,5789991


## TASK 5: Calculate worldwide COVID testing positive ratio

The goal of task 5 is to get the total confirmed and tested cases worldwide, and try to figure the overall positive ratio using `confirmed cases / tested cases`


In [67]:
# Get the total confirmed cases worldwide
Total_confirmed_cases <- sum(covid_df_csv$confirmed)
Total_confirmed_cases
# Get the total tested cases worldwide
Total_tested_cases <- sum(covid_df_csv$tested)
Total_tested_cases
# Get the positive ratio (confirmed / tested)
Positive_ratio <- Total_confirmed_cases/Total_tested_cases
Positive_ratio

[1] 431191348

[1] 5392023544

[1] 0.07996837

## TASK 6: Get a country list which reported their testing data 

The goal of task 6 is to get a catalog or sorted list of countries who have reported their COVID-19 testing data


In [1]:
# Get the `country` column

# Check its class (should be Factor)

# Conver the country column into character so that you can easily sort them

# Sort the countries AtoZ

# Sort the countries ZtoA

# Print the sorted ZtoA list


ERROR: Error in eval(expr, envir, enclos): object 'covid_df_csv' not found


In [2]:
# Get the `country` column
covid_df_csv$country

ERROR: Error in eval(expr, envir, enclos): object 'covid_df_csv' not found


In [ ]:
# Check its class (should be Factor)
class(covid_df_csv$country)

## TASK 7: Identify countries names with a specific pattern

The goal of task 7 is using a regular expression to find any countires start with `United`


In [ ]:
# Use a regular expression `United.+` to find matches

# Print the matched country names


## TASK 8: Pick two countries you are interested, and then review their testing data

The goal of task 8 is to compare the COVID-19 test data between two countires, you will need to select two rows from the dataframe, and select `country`, `confirmed`, `confirmed-population-ratio` columns


In [ ]:
# Select a subset (should be only one row) of data frame based on a selected country name and columns


# Select a subset (should be only one row) of data frame based on a selected country name and columns


## TASK 9: Compare which one of the selected countries has a larger ratio of confirmed cases to population

The goal of task 9 is to find out which country you have selected before has larger ratio of confirmed cases to population, which may indicate that country has higher COVID-19 infection risk


In [ ]:
# Use if-else statement
# if (check which confirmed.population value is greater) {
#    print()
# } else {
#    print()
# }


## TASK 10: Find countries with confirmed to population ratio rate less than a threshold

The goal of task 10 is to find out which countries have the confirmed to population ratio less than 1%, it may indicate the risk of those countries are relatively low


In [ ]:
# Get a subset of any countries with `confirmed.population.ratio` less than the threshold
